# Data Preparation

### Task
Parse the json **nyc_geo.json** into the dataframe with the following columns:
- Borough
- Neighborhood
- Latitude
- Longitude

In [311]:
from IPython.display import JSON
import pandas as pd
import numpy as np
import json
import geopy.distance
import requests as re
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#KEYS!
fs_api_key = os.environ["FS_API_KEY"]
google_api = os.environ["GOOGLE_API"]

In [14]:
#process neighborhoods data into dataframe
file = open('nyc_geo.json')
jsondata = json.load(file)

In [15]:
JSON(jsondata)

<IPython.core.display.JSON object>

In [16]:
#to dict first
dictdata = []
for i in range(len(jsondata['features'])):
    borough = jsondata['features'][i]['properties']['borough']
    hood = jsondata['features'][i]['properties']['name']
    latitude = jsondata['features'][i]['geometry']['coordinates'][1]
    longitude = jsondata['features'][i]['geometry']['coordinates'][0]
    dataentry = [hood, borough, latitude, longitude]
    dictdata.append(dataentry)

In [17]:
columns = ['neighborhood', 'borough', 'nb latitude', 'nb longitude']
df = pd.DataFrame(dictdata, columns=columns)

In [18]:
df.head()

,neighborhood,borough,nb latitude,nb longitude
0,Wakefield,Bronx,40.894705,-73.847201
1,Co-op City,Bronx,40.874294,-73.829939
2,Eastchester,Bronx,40.887556,-73.827806
3,Fieldston,Bronx,40.895437,-73.905643
4,Riverdale,Bronx,40.890834,-73.912585


### Task
Use different data sources and APIs to collect information about the neigborhoods that can be used for segmentation.

#### FOURSQUARE

In [27]:
#initialize FS API
headers = {

    "Accept": "application/json",

    "Authorization": fs_api_key

}
url="https://api.foursquare.com/v3/places/search"
radius = "&radius=500"
limit = "&limit=50"
#center is currently the coordinates of the first entry, wakefield i think
center="40.894705,-73.847201"

In [28]:
#get venues list given a set of coordinates from FS API
def get_venues(coords):
    stripcoords = coords.replace(" ", "")
    url_keys_append = "?fields=name,location,geocodes,categories&ll=" + stripcoords + radius + limit
    requrl = url + url_keys_append
    res = re.request("GET", requrl, headers=headers)
    print(res.status_code)
    dataset = res.json()
    return dataset;

In [29]:
jsonres = get_venues(center)

200


In [39]:
#jsonres['results'][6]['location']['neighborhood'][0]
JSON(jsonres['results'])

<IPython.core.display.JSON object>

In [25]:
#what i want for from each item
venues_data = []
for i in range(len(jsonres['results'])):
    print(i)
    venue_type = jsonres['results'][i]['categories'][0]['name']
    venue_lat = jsonres['results'][i]['geocodes']['main']['latitude']
    venue_lon = jsonres['results'][i]['geocodes']['main']['longitude']
    new_venue_item = [venue_type, venue_lat, venue_lon]
    venues_data.append(new_venue_item)

0
1
2
3
4
5
6
7
8
9
10


In [26]:
venues_data

[['Ice Cream Parlor', 40.893585, -73.843692],
 ['Drugstore', 40.896423, -73.844714],
 ['Drugstore', 40.896421, -73.844802],
 ['Deli', 40.896768, -73.844307],
 ['Ice Cream Parlor', 40.890433, -73.848571],
 ['American Restaurant', 40.898173, -73.850254],
 ['Fast Food Restaurant', 40.890473, -73.849142],
 ['Other Great Outdoors', 40.891511, -73.844493],
 ['Car Wash and Detail', 40.89121, -73.84987],
 ['American Restaurant', 40.890909, -73.848983],
 ['Laundromat', 40.890576, -73.849321]]

#### GOOGLE

In [31]:
#GOOGLE API
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
params= "location=40.894705%2C-73.847201&radius=500&key=" + google_api

payload={}
headers={}



In [34]:
goourl = url + params

In [35]:
response = re.request("GET", goourl, headers=headers, data=payload)

print(response.status_code)

{
   "html_attributions" : [],
   "next_page_token" : "Aap_uEDuZLKGvo2l4OkT0tv0LHfzYxTErmm-3tSg7gB8yeoyGK5pyuye2SYVEBZoOAa6p-_nttxO1u5MT6W7hIS1hB53W1yUzEtcSVsU5te9BHTTgER0PTSwPNTvCJK-7is8LC1WhTzXr3n_MrjltlQ685-vr8Z_Q5x1XoDq9wk65K-Y3JKrFhzdKr61MseOrUf-QbinoqqiFnXMdkVbBByBL6QzP34w88UNsvDq3oDAR_iMnBhhPPR3xOI0EzUhq5xui-mirSwOqOe1HRXIrFFHK_fMWKt41u15iH5VmJaZNXBcKiUphSADodQbcjMkYJTwD3qKXZcTkx5M8O6oq9fQ_KQcLzg2IvyCiA2un234VPxGZg10he_7MwtNkgqymEVdqVq4YZdLbxG_EZ8N5YBIzOp4I-0nZlo9F5bq0X0IwOh8",
   "results" : [
      {
         "geometry" : {
            "location" : {
               "lat" : 40.7127753,
               "lng" : -74.0059728
            },
            "viewport" : {
               "northeast" : {
                  "lat" : 40.91757705070789,
                  "lng" : -73.70027206817629
               },
               "southwest" : {
                  "lat" : 40.47739906045452,
                  "lng" : -74.25908991427882
               }
            }
         },
         "icon" : "

In [36]:
goo_result = response.json()

In [51]:
JSON(goo_result['results'])

<IPython.core.display.JSON object>

### HOUSING PRICES DATA

In [566]:
#get housing prices data
febexcel = pd.read_excel('nyc_housing_prices_feb_2021.xlsx')
julcsv = pd.read_csv('nyc_housing_prices_jul_2020.csv')

In [567]:
#working on these columns a lot
process_cols = ['studio', '1_bedroom', '2_bedroom']

In [568]:
#remove commas and dollar signs so we can count mean
for i in process_cols:
    febexcel[i] = febexcel[i].str.replace(',', '')
    febexcel[i] = febexcel[i].str.replace('$', '')

for i in julcsv.columns[2:6]:
    julcsv[i] = julcsv[i].str.replace(',', '')
    julcsv[i] = julcsv[i].str.replace('$', '')

In [569]:
#realign excel table data because it is not aligned in the end (check febexcel.tail())
#take it all apart first. im sure theres a more elegant way
febexcelstud = febexcel[['neighborhood','studio']].copy()
febexcelonebd = febexcel[['neighborhood.1','1_bedroom']].copy()
febexcelonebd.rename(columns={"neighborhood.1":"neighborhood"}, inplace=True)
febexceltwobd = febexcel[['neighborhood.2', '2_bedroom']].copy()
febexceltwobd.rename(columns={"neighborhood.2":"neighborhood"}, inplace=True)

In [570]:
#drop null values from datasets, otherwise merge will give too many results
febexcelstud.dropna(subset=['studio'], inplace=True)
febexcelonebd.dropna(subset=['1_bedroom'], inplace=True)
febexceltwobd.dropna(subset=['2_bedroom'], inplace=True)

In [571]:
#make existing values floats
febexcelstud['studio'] = febexcelstud['studio'].astype(float)
febexcelonebd['1_bedroom'] = febexcelonebd['1_bedroom'].astype(float)
febexceltwobd['2_bedroom'] = febexceltwobd['2_bedroom'].astype(float)

In [572]:
#merge separated tables on neighborhood
#merge on right because the second and third columns had more values
feb_merged_one = pd.merge(febexcelstud, febexcelonebd, how='right', on='neighborhood')
feb_merged = pd.merge(feb_merged_one, febexceltwobd, how='right', on='neighborhood')

In [573]:
#fill nans with column mean
for i in process_cols:
    feb_merged[i].fillna((feb_merged[i].mean()), inplace=True)

In [574]:
#make new column called mean based on the other values
feb_merged['mean'] = feb_merged.mean(axis=1)

In [575]:
feb_merged.head()

,neighborhood,studio,1_bedroom,2_bedroom,mean
0,Upper West Side,1975.0,2750.0,4350.0,3025.000000
1,Bedford-Stuyvesant,1650.0,1975.0,2200.0,1941.666667
2,Bushwick,1731.0,2150.0,2100.0,1993.666667
3,Upper East Side,1898.0,2395.0,3295.0,2529.333333
4,Williamsburg,2491.0,2500.0,2584.0,2525.000000


In [576]:
#GREAT SUCCESS! save to file just in case
feb_merged.to_excel('worked_feb.xlsx')

In [577]:
#on to the july 2021 dataset
#spelling error in column name
julcsv.rename(columns={"neigborhood":"neighborhood"}, inplace=True)

In [578]:
#drop borough and 3_bedroom because we dont have that info in the prev dataset
julcsv.drop(columns=['borough', '3_bedroom'], inplace=True)

In [579]:
# naughty red hook is missing a val
julcsv.at[50, 'studio'] = 0

In [580]:
#make those values float
for i in process_cols:
    julcsv[i] = julcsv[i].astype(float)

In [581]:
#red hook again
julcsv.at[50, 'studio'] = julcsv['studio'].mean()

In [582]:
#get mean for this dataset as well
julcsv['mean'] = julcsv.mean(axis=1)

In [583]:
#save
julcsv.to_excel('worked_july.xlsx')

In [584]:
prices = pd.merge(feb_merged, julcsv, how='outer', on='neighborhood')

In [585]:
pd.set_option('display.max_rows', prices.shape[0]+1)

In [588]:

prices.sort_values(by=['neighborhood'], inplace=True)

AttributeError: 'NoneType' object has no attribute 'reset_index'

In [592]:
prices.reset_index(drop=True, inplace=True)


,neighborhood,studio_x,1_bedroom_x,2_bedroom_x,mean_x,studio_y,1_bedroom_y,2_bedroom_y,mean_y
0,Allerton,1450.000000,1625.000000,1800.0,1625.000000,NaN,NaN,NaN,NaN
1,Arden Heights,2019.386861,2028.437158,2250.0,2099.274673,NaN,NaN,NaN,NaN
2,Astoria,1575.000000,1875.000000,2200.0,1883.333333,NaN,NaN,NaN,NaN
3,Auburndale,1795.000000,1780.000000,2175.0,1916.666667,NaN,NaN,NaN,NaN
4,Bath Beach,3000.000000,1575.000000,1825.0,2133.333333,NaN,NaN,NaN,NaN
5,Battery Park City,2200.000000,2733.000000,4431.0,3121.333333,2823.0,3573.0,6859.0,4418.333333
6,Bay Ridge,2038.000000,1750.000000,2109.0,1965.666667,NaN,NaN,NaN,NaN
7,Bayside,1850.000000,1850.000000,2300.0,2000.000000,NaN,NaN,NaN,NaN
8,Bedford Park,1413.000000,1525.000000,2100.0,1679.333333,NaN,NaN,NaN,NaN
9,Bedford-Stuyvesant,1650.000000,1975.000000,2200.0,1941.666667,2044.0,2297.0,2611.0,2317.333333


In [600]:
#if left dataset has nan, take it from the right dataset
for i in range(len(prices)):
    if (prices.at[i, 'studio_x'] != prices.at[i, 'studio_x']):
             prices.at[i, 'mean_x'] = prices.at[i, 'mean_y']

In [602]:
prices.drop(columns=['studio_x', '1_bedroom_x', '2_bedroom_x', 'studio_y', '1_bedroom_y', '2_bedroom_y', 'mean_y'], inplace=True)

In [603]:
prices.head()

,neighborhood,mean_x
0,Allerton,1625.000000
1,Arden Heights,2099.274673
2,Astoria,1883.333333
3,Auburndale,1916.666667
4,Bath Beach,2133.333333


In [604]:
prices.to_excel('housing_means.xlsx')

### Task

Visualize the neigborhoods in the graph.